In [28]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib notebook
%matplotlib inline
import pywt
import numpy as np
import pandas as pd
import seaborn as sns

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score

import scipy.stats as ss

import scipy.io as sio
import math
import sys
!{sys.executable} -m pip install siml
from siml.sk_utils import *
from siml.signal_analysis_utils import *
import sys
!{sys.executable} -m pip install sensormotion
import sensormotion as sm
import sys
!{sys.executable} -m pip install tensorflow
import tensorflow as tf
from scipy.fftpack import fft
from scipy.signal import welch

import time
import datetime as dt
from collections import defaultdict, Counter

import warnings
warnings.filterwarnings("ignore")

  Using cached tensorflow-2.3.1-cp38-cp38-win_amd64.whl (342.5 MB)
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
Processing c:\users\infinigru\appdata\local\pip\cache\wheels\a0\16\9c\5473df82468f958445479c59e784896fa24f4a5fc024b0f501\termcolor-1.1.0-py3-none-any.whl
  Using cached protobuf-3.13.0-py2.py3-none-any.whl (438 kB)
  Using cached grpcio-1.32.0-cp38-cp38-win_amd64.whl (2.6 MB)
  Using cached absl_py-0.10.0-py3-none-any.whl (127 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached google_auth-1.22.0-py2.py3-none-any.whl (114 kB)
  Using cached Markdown-3.2.2-py3-none-any.whl (88 kB)
  Using cached google_auth_oauthlib-0.4.1-py2.p

ImportError: Traceback (most recent call last):
  File "C:\Users\InfiniGru\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 64, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: The specified module could not be found.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [24]:
train_time_series = pd.read_csv("C:\\Users\\InfiniGru\\Documents\\GitHub\\EdX-Python-for-Research\\Final Project\\train_time_series.csv")
train_time_series.head()

,Unnamed: 0,timestamp,UTC time,accuracy,x,y,z
0,20586,1565109930787,2019-08-06T16:45:30.787,unknown,-0.006485,-0.934860,-0.069046
1,20587,1565109930887,2019-08-06T16:45:30.887,unknown,-0.066467,-1.015442,0.089554
2,20588,1565109930987,2019-08-06T16:45:30.987,unknown,-0.043488,-1.021255,0.178467
3,20589,1565109931087,2019-08-06T16:45:31.087,unknown,-0.053802,-0.987701,0.068985
4,20590,1565109931188,2019-08-06T16:45:31.188,unknown,-0.054031,-1.003616,0.126450


In [15]:
time_stamp = train_time_series["timestamp"]
x = train_time_series["x"]
y = train_time_series["y"]
z = train_time_series["z"]

In [17]:
train_labels = pd.read_csv("C:\\Users\\InfiniGru\\Documents\\GitHub\\EdX-Python-for-Research\\Final Project\\train_labels.csv")
train_labels.head()

,Unnamed: 0,timestamp,UTC time,label
0,20589,1565109931087,2019-08-06T16:45:31.087,1
1,20599,1565109932090,2019-08-06T16:45:32.090,1
2,20609,1565109933092,2019-08-06T16:45:33.092,1
3,20619,1565109934094,2019-08-06T16:45:34.094,1
4,20629,1565109935097,2019-08-06T16:45:35.097,1


In [22]:
standing = train_labels[train_labels["label"]==1]
walking  = train_labels[train_labels["label"]==2]
stairs_down  = train_labels[train_labels["label"]==3]
stairs_up  = train_labels[train_labels["label"]==4]
print(standing)

     Unnamed: 0      timestamp                 UTC time  label
0         20589  1565109931087  2019-08-06T16:45:31.087      1
1         20599  1565109932090  2019-08-06T16:45:32.090      1
2         20609  1565109933092  2019-08-06T16:45:33.092      1
3         20619  1565109934094  2019-08-06T16:45:34.094      1
4         20629  1565109935097  2019-08-06T16:45:35.097      1
5         20639  1565109936099  2019-08-06T16:45:36.099      1
6         20649  1565109937101  2019-08-06T16:45:37.101      1
7         20659  1565109938103  2019-08-06T16:45:38.103      1
8         20669  1565109939106  2019-08-06T16:45:39.106      1
134       21929  1565110065487  2019-08-06T16:47:45.487      1
135       21939  1565110066490  2019-08-06T16:47:46.490      1
136       21949  1565110067492  2019-08-06T16:47:47.492      1
137       21959  1565110068494  2019-08-06T16:47:48.494      1
138       21969  1565110069497  2019-08-06T16:47:49.497      1
139       21979  1565110070499  2019-08-06T16:47:50.499